In [1]:
from os import path as osp
import numpy as np
import tensorflow as tf
import sonnet as snt
from attrdict import AttrDict

from evaluation import make_fig, make_logger

from data import load_data, tensors_from_data
from mnist_model import SeqAIRonMNIST

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
% matplotlib inline

In [2]:
learning_rate = 1e-4
n_steps = 3

results_dir = '../results/seq_grad_inspect'
run_name = 'no_baseline'

logdir = osp.join(results_dir, run_name)
checkpoint_name = osp.join(logdir, 'model.ckpt')
axes = {'imgs': 1, 'labels': 0, 'nums': 1, 'coords': 1}

In [3]:
n_timesteps = 1
batch_size = 64

num_steps_prior = AttrDict(
    anneal='exp',
    init=1.,
    final=1e-5,
    steps_div=1e4,
    steps=1e5,
    hold_init=1e3,
    analytic=False
)


appearance_prior = AttrDict(loc=0., scale=1.)
where_scale_prior = AttrDict(loc=0., scale=1.)
where_shift_prior = AttrDict(loc=0., scale=1.)

num_steps_prior = None
appearance_prior = None
where_scale_prior = None
where_shift_prior = None

use_reinforce = True
step_bias = .75
transform_var_bias = .5
output_multiplier = .5

l2_weight = 0. #1e-5

supervised_nums = False

In [4]:
# valid_data = load_data('small_seq_mnist_validation.pickle')
# train_data = load_data('small_seq_mnist_train.pickle')

In [5]:
valid_data = load_data('mnist_validation.pickle')
train_data = load_data('mnist_train.pickle')

# del axes['coords']
def fix(d):
    d.imgs = d.imgs[np.newaxis]
    d.nums = d.nums.T
    for k in d:
        print k, d[k].shape
    return d

valid_data, train_data = [fix(d) for d in (valid_data, train_data)]

imgs (1, 10000, 50, 50)
labels (10000, 2)
nums (1, 10000, 3)
imgs (1, 60000, 50, 50)
labels (60000, 2)
nums (1, 60000, 3)


In [6]:
tf.reset_default_graph()
train_tensors = tensors_from_data(train_data, batch_size, axes, shuffle=True)
valid_tensors = tensors_from_data(valid_data, batch_size, axes, shuffle=False)
x, valid_x = train_tensors['imgs'], valid_tensors['imgs']
y, test_y = train_tensors['nums'], valid_tensors['nums']
    
n_hiddens = 32 * 8
n_layers = 2
n_hiddens = [n_hiddens] * n_layers
    
n_timesteps = tf.Variable(n_timesteps, trainable=False, dtype=tf.int32, name='n_timesteps')
seq_x = x[:n_timesteps]
seq_y = y[:n_timesteps]
air = SeqAIRonMNIST(seq_x,
                max_steps=n_steps,
                inpt_encoder_hidden=n_hiddens,
                glimpse_encoder_hidden=n_hiddens,
                glimpse_decoder_hidden=n_hiddens,
                transform_estimator_hidden=n_hiddens,
                steps_pred_hidden=[128, 64],
                baseline_hidden=[256, 128],
                transform_var_bias=transform_var_bias,
                step_bias=step_bias,
                output_multiplier=output_multiplier
)

In [7]:
train_step, global_step = air.train_step(learning_rate, l2_weight, appearance_prior, where_scale_prior,
                            where_shift_prior, num_steps_prior, use_reinforce=use_reinforce,
                            decay_rate=None, nums=seq_y, supervised_nums=supervised_nums)

reinf (?, 64) (?, 64, 1)
model 34
lstm/w_gates:0 [512, 1024]
SeqAIRonMNIST/SeqAIRCell/StochasticTransformParam/MLP/linear_2/b:0 [8]
SeqAIRonMNIST/SeqAIRCell/StepsPredictor/MLP/linear_2/w:0 [64, 1]
SeqAIRonMNIST/SeqAIRCell/StochasticTransformParam/MLP/linear_2/w:0 [256, 8]
SeqAIRonMNIST/SeqAIRCell/StepsPredictor/MLP/linear_2/b:0 [1]
lstm/b_gates:0 [1024]
SeqAIRonMNIST/SeqAIRCell/Encoder/MLP/linear_1/w:0 [256, 256]
SeqAIRonMNIST/SeqAIRCell/Decoder/MLP/linear/w:0 [50, 256]
SeqAIRonMNIST/SeqAIRCell/StochasticTransformParam/MLP/linear/b:0 [256]
SeqAIRonMNIST/lstm_initial_state_0/w:0 [1, 256]
SeqAIRonMNIST/SeqAIRCell/StochasticTransformParam/MLP/linear/w:0 [256, 256]
SeqAIRonMNIST/SeqAIRCell/Decoder/MLP/linear/b:0 [256]
SeqAIRonMNIST/SeqAIRCell/Encoder/MLP/linear/w:0 [2500, 256]
SeqAIRonMNIST/SeqAIRCell/Decoder/MLP/linear_2/w:0 [256, 400]
SeqAIRonMNIST/SeqAIRCell/Decoder/MLP/linear_1/b:0 [256]
SeqAIRonMNIST/SeqAIRCell/StepsPredictor/MLP/linear/b:0 [128]
SeqAIRonMNIST/SeqAIRCell/Decoder/MLP/l

/Users/adam/anaconda2/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Gradient for variable SeqAIRonMNIST/what_init is None
Gradient for variable SeqAIRonMNIST/where_init is None


In [8]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
    
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [9]:
step_prob = air.num_steps_distrib.prob()
mean_prob = tf.reduce_mean(step_prob, (0, 1))
step_entropy = step_prob * tf.log(step_prob)
step_entropy = -tf.reduce_sum(step_entropy)

In [10]:
def compute_grad(y, x):
    g = tf.reduce_mean(tf.gradients(y, x)[0], (0, 1))
    norm = 1.#tf.sqrt(tf.reduce_sum(tf.square(g), -1, keep_dims=True))
    return g / norm

total_grad = compute_grad(air.opt_loss, step_prob)

grads = [total_grad]
names = ['total']

if air.use_prior:
    kl_grad = compute_grad(air.prior_loss.value, step_prob)
    grads.append(kl_grad)
    names.append('kl')
    
if use_reinforce:
    reinforce_grad = compute_grad(air.reinforce_loss, step_prob)
    grads.append(reinforce_grad)
    names.append('reinforce')
    
grads.append(mean_prob)
names.append('prob')


for g, name in zip(grads, names):
    step_grads = tf.unstack(g, axis=-1)
    for i, step_grad in enumerate(step_grads):
        tf.summary.scalar('prob_grad/{}_{}'.format(name, i), step_grad)

In [11]:
print grads
print names

[<tf.Tensor 'div:0' shape=(4,) dtype=float32>, <tf.Tensor 'div_1:0' shape=(4,) dtype=float32>, <tf.Tensor 'Mean_1:0' shape=(4,) dtype=float32>]
['total', 'reinforce', 'prob']


In [12]:
# a, d = sess.run([grads, mean_prob])
# for i in a:
#     print i
# print d

In [13]:
all_summaries = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter(logdir, sess.graph)
saver = tf.train.Saver()

In [14]:
n_train_samples = train_data['imgs'].shape[axes['imgs']]
n_valid_samples = valid_data['imgs'].shape[axes['imgs']]
n_train_samples, n_valid_samples = [i//100 for i in (n_train_samples, n_valid_samples)] 
log = make_logger(air, sess, summary_writer, train_tensors, n_train_samples, valid_tensors, n_valid_samples)

In [15]:
n_train_samples, n_valid_samples

(600, 100)

In [16]:
# log(0)

In [17]:
# tensors = {
#     'loss': air.loss.value,
#     'imp_weight': .5*tf.reduce_mean((air.reinforce_imp_weight)**2),
#     'baseline_loss': air.baseline_loss,
#     'baseline': tf.reduce_mean(air.baseline)
# }

In [18]:
# output = sess.run(tensors)
# for o, v in output.iteritems():
#     print '{}: {}'.format(o, v)

In [ ]:
* template and coord storing in data.create_mnist
* load_data returns an AttrDict
* 

In [20]:
train_itr = sess.run(global_step)
print 'Starting training at iter = {}'.format(train_itr)

if train_itr == 0:
#     if air.use_reinforce:
#         sess.run(air._baseline_train_step)
    log(0)
    
while train_itr < 1e6:
        
    train_itr, _ = sess.run([global_step, train_step])
    
#     if train_itr % 100 == 0:
#     if (train_itr % 1000) < 100:
    if train_itr % 50 == 0:
        summaries = sess.run(all_summaries)
        summary_writer.add_summary(summaries, train_itr)
        
    if train_itr % 100 == 0:
        log(train_itr)
        
    if train_itr % 5000 == 0:
        saver.save(sess, checkpoint_name, global_step=train_itr)
#         make_fig(air, sess, logdir, train_itr)    

Starting training at iter = 0
Step 0, Data train loss = -118.6387, imp_weight = -118.6387, opt_loss = 40.1163, reinforce_loss = 158.7550, rec_loss = -118.6387, num_step = 1.3524, num_step_acc = 0.2222, eval time = 0.2532s
Step 0, Data test loss = -29.1283, imp_weight = -29.1283, opt_loss = 9.4869, reinforce_loss = 38.6153, rec_loss = -29.1283, num_step = 1.4062, num_step_acc = 0.1719, eval time = 0.09908s

Step 100, Data train loss = -199.0291, imp_weight = -199.0291, opt_loss = 66.0636, reinforce_loss = 265.0926, rec_loss = -199.0291, num_step = 1.2274, num_step_acc = 0.2153, eval time = 0.1685s
Step 100, Data test loss = -150.9201, imp_weight = -150.9201, opt_loss = 49.1561, reinforce_loss = 200.0762, rec_loss = -150.9201, num_step = 1.3281, num_step_acc = 0.1875, eval time = 0.01688s

Step 200, Data train loss = -263.1973, imp_weight = -263.1973, opt_loss = 68.8205, reinforce_loss = 332.0178, rec_loss = -263.1973, num_step = 1.2378, num_step_acc = 0.2049, eval time = 0.1832s
Step 20

Step 2100, Data train loss = -507.3460, imp_weight = -507.3460, opt_loss = 162.0468, reinforce_loss = 669.3929, rec_loss = -507.3460, num_step = 1.7378, num_step_acc = 0.1823, eval time = 0.1924s
Step 2100, Data test loss = -439.9178, imp_weight = -439.9178, opt_loss = 163.5695, reinforce_loss = 603.4873, rec_loss = -439.9178, num_step = 1.5625, num_step_acc = 0.2656, eval time = 0.01952s

Step 2200, Data train loss = -506.4790, imp_weight = -506.4790, opt_loss = 145.8249, reinforce_loss = 652.3039, rec_loss = -506.4790, num_step = 1.7448, num_step_acc = 0.1962, eval time = 0.1837s
Step 2200, Data test loss = -528.3838, imp_weight = -528.3838, opt_loss = 158.8640, reinforce_loss = 687.2477, rec_loss = -528.3838, num_step = 1.8594, num_step_acc = 0.2812, eval time = 0.01929s

Step 2300, Data train loss = -489.9593, imp_weight = -489.9593, opt_loss = 158.1847, reinforce_loss = 648.1440, rec_loss = -489.9593, num_step = 1.6562, num_step_acc = 0.2135, eval time = 0.1935s
Step 2300, Data te

Step 4200, Data train loss = -529.2955, imp_weight = -529.2955, opt_loss = 171.5116, reinforce_loss = 700.8071, rec_loss = -529.2955, num_step = 1.6823, num_step_acc = 0.2135, eval time = 0.1794s
Step 4200, Data test loss = -444.4440, imp_weight = -444.4440, opt_loss = 124.4225, reinforce_loss = 568.8665, rec_loss = -444.4440, num_step = 1.5312, num_step_acc = 0.2188, eval time = 0.01897s

Step 4300, Data train loss = -497.1604, imp_weight = -497.1604, opt_loss = 155.8989, reinforce_loss = 653.0593, rec_loss = -497.1604, num_step = 1.6163, num_step_acc = 0.2188, eval time = 0.1787s
Step 4300, Data test loss = -452.6310, imp_weight = -452.6310, opt_loss = 150.1467, reinforce_loss = 602.7777, rec_loss = -452.6310, num_step = 1.5000, num_step_acc = 0.1875, eval time = 0.01958s

Step 4400, Data train loss = -510.2068, imp_weight = -510.2068, opt_loss = 163.8861, reinforce_loss = 674.0929, rec_loss = -510.2068, num_step = 1.7066, num_step_acc = 0.2101, eval time = 0.1831s
Step 4400, Data te

Step 6300, Data train loss = -499.5271, imp_weight = -499.5271, opt_loss = 159.7515, reinforce_loss = 659.2786, rec_loss = -499.5271, num_step = 1.6458, num_step_acc = 0.1892, eval time = 0.1806s
Step 6300, Data test loss = -470.4097, imp_weight = -470.4097, opt_loss = 145.5066, reinforce_loss = 615.9163, rec_loss = -470.4097, num_step = 1.6250, num_step_acc = 0.2188, eval time = 0.01995s

Step 6400, Data train loss = -528.9655, imp_weight = -528.9655, opt_loss = 170.6001, reinforce_loss = 699.5657, rec_loss = -528.9655, num_step = 1.6545, num_step_acc = 0.2135, eval time = 0.182s
Step 6400, Data test loss = -511.0793, imp_weight = -511.0793, opt_loss = 218.9889, reinforce_loss = 730.0682, rec_loss = -511.0793, num_step = 1.5469, num_step_acc = 0.2969, eval time = 0.01881s

Step 6500, Data train loss = -502.4463, imp_weight = -502.4463, opt_loss = 156.0632, reinforce_loss = 658.5094, rec_loss = -502.4463, num_step = 1.6267, num_step_acc = 0.1997, eval time = 0.1846s
Step 6500, Data tes

Step 8400, Data train loss = -508.5145, imp_weight = -508.5145, opt_loss = 167.4895, reinforce_loss = 676.0039, rec_loss = -508.5145, num_step = 1.6215, num_step_acc = 0.1892, eval time = 0.174s
Step 8400, Data test loss = -456.2158, imp_weight = -456.2158, opt_loss = 150.4776, reinforce_loss = 606.6934, rec_loss = -456.2158, num_step = 1.6250, num_step_acc = 0.1250, eval time = 0.01928s

Step 8500, Data train loss = -514.5843, imp_weight = -514.5843, opt_loss = 160.0290, reinforce_loss = 674.6133, rec_loss = -514.5843, num_step = 1.6910, num_step_acc = 0.1806, eval time = 0.1772s
Step 8500, Data test loss = -494.5698, imp_weight = -494.5698, opt_loss = 191.1527, reinforce_loss = 685.7225, rec_loss = -494.5698, num_step = 1.6094, num_step_acc = 0.2656, eval time = 0.01968s

Step 8600, Data train loss = -497.1361, imp_weight = -497.1361, opt_loss = 156.7696, reinforce_loss = 653.9057, rec_loss = -497.1361, num_step = 1.5799, num_step_acc = 0.2031, eval time = 0.2323s
Step 8600, Data tes

...

Step 73500, Data train loss = -523.5993, imp_weight = -523.5993, opt_loss = 165.0930, reinforce_loss = 688.6923, rec_loss = -523.5993, num_step = 1.6372, num_step_acc = 0.2188, eval time = 0.1788s
Step 73500, Data test loss = -543.5380, imp_weight = -543.5380, opt_loss = 209.0560, reinforce_loss = 752.5940, rec_loss = -543.5380, num_step = 1.7344, num_step_acc = 0.2188, eval time = 0.01903s

Step 73600, Data train loss = -534.2201, imp_weight = -534.2201, opt_loss = 161.9961, reinforce_loss = 696.2161, rec_loss = -534.2201, num_step = 1.6476, num_step_acc = 0.1892, eval time = 0.1843s
Step 73600, Data test loss = -493.6873, imp_weight = -493.6873, opt_loss = 144.9930, reinforce_loss = 638.6802, rec_loss = -493.6873, num_step = 1.6406, num_step_acc = 0.1875, eval time = 0.02007s

Step 73700, Data train loss = -520.7867, imp_weight = -520.7867, opt_loss = 171.2896, reinforce_loss = 692.0763, rec_loss = -520.7867, num_step = 1.6476, num_step_acc = 0.1944, eval time = 0.1839s
Step 73700, D

Step 75600, Data train loss = -527.3914, imp_weight = -527.3914, opt_loss = 165.5201, reinforce_loss = 692.9116, rec_loss = -527.3914, num_step = 1.6649, num_step_acc = 0.1806, eval time = 0.1835s
Step 75600, Data test loss = -475.4633, imp_weight = -475.4633, opt_loss = 140.0880, reinforce_loss = 615.5512, rec_loss = -475.4633, num_step = 1.5781, num_step_acc = 0.2344, eval time = 0.01902s

Step 75700, Data train loss = -529.4142, imp_weight = -529.4142, opt_loss = 161.8224, reinforce_loss = 691.2365, rec_loss = -529.4142, num_step = 1.6128, num_step_acc = 0.2240, eval time = 0.1819s
Step 75700, Data test loss = -491.8780, imp_weight = -491.8780, opt_loss = 138.7210, reinforce_loss = 630.5990, rec_loss = -491.8780, num_step = 1.7344, num_step_acc = 0.1250, eval time = 0.0195s

Step 75800, Data train loss = -521.9401, imp_weight = -521.9401, opt_loss = 156.1509, reinforce_loss = 678.0911, rec_loss = -521.9401, num_step = 1.7535, num_step_acc = 0.1615, eval time = 0.1882s
Step 75800, Da

Step 77700, Data train loss = -524.4464, imp_weight = -524.4464, opt_loss = 171.0422, reinforce_loss = 695.4885, rec_loss = -524.4464, num_step = 1.5938, num_step_acc = 0.2257, eval time = 0.1968s
Step 77700, Data test loss = -468.7150, imp_weight = -468.7150, opt_loss = 122.5704, reinforce_loss = 591.2854, rec_loss = -468.7150, num_step = 1.7344, num_step_acc = 0.1094, eval time = 0.02342s

Step 77800, Data train loss = -535.7700, imp_weight = -535.7700, opt_loss = 168.3706, reinforce_loss = 704.1406, rec_loss = -535.7700, num_step = 1.7083, num_step_acc = 0.2257, eval time = 0.1806s
Step 77800, Data test loss = -471.8835, imp_weight = -471.8835, opt_loss = 119.6674, reinforce_loss = 591.5508, rec_loss = -471.8835, num_step = 1.5781, num_step_acc = 0.1719, eval time = 0.01873s

Step 77900, Data train loss = -531.2885, imp_weight = -531.2885, opt_loss = 168.0867, reinforce_loss = 699.3751, rec_loss = -531.2885, num_step = 1.6458, num_step_acc = 0.1927, eval time = 0.1776s
Step 77900, D

Step 79800, Data train loss = -522.0952, imp_weight = -522.0952, opt_loss = 163.6660, reinforce_loss = 685.7613, rec_loss = -522.0952, num_step = 1.6007, num_step_acc = 0.2222, eval time = 0.2287s
Step 79800, Data test loss = -482.1916, imp_weight = -482.1916, opt_loss = 132.1759, reinforce_loss = 614.3676, rec_loss = -482.1916, num_step = 1.5781, num_step_acc = 0.1250, eval time = 0.0247s

Step 79900, Data train loss = -520.0135, imp_weight = -520.0135, opt_loss = 166.2109, reinforce_loss = 686.2244, rec_loss = -520.0135, num_step = 1.6441, num_step_acc = 0.1979, eval time = 0.2628s
Step 79900, Data test loss = -509.9578, imp_weight = -509.9578, opt_loss = 146.7956, reinforce_loss = 656.7534, rec_loss = -509.9578, num_step = 1.6875, num_step_acc = 0.2344, eval time = 0.02842s



KeyboardInterrupt: 

In [ ]:
gt, imgs = sess.run([x, air.canvas])

In [ ]:
nt = imgs.shape[0]
fig, axes = plt.subplots(2, nt, figsize=(nt*2, 4), sharex=True, sharey=True)
axes = axes.reshape((2, nt))
n = np.random.randint(imgs.shape[1])
for t, ax in enumerate(axes.T):
    ax[0].imshow(imgs[t, n], cmap='gray', vmin=0., vmax=1.)
    ax[1].imshow(gt[t, n], cmap='gray', vmin=0., vmax=1.)
    for a in ax:
        a.grid(False)